In [ ]:
import mne
import mne_bids
from mne_bids import BIDSPath
import os, os.path as op
import nilearn
import matplotlib.pyplot as plt
import numpy as np
import nih2mne

#from nilearn import *
from nih2mne.utilities.bids_helpers import get_mri_dict
n_jobs = 10 #Number of parrallel operations

In [ ]:
# first let's set up directories

bids_root = '/Users/nugenta/meg_workshop_data'
#bids_root = op.join('/data',os.environ['USER'], 'meg_workshop_data') 
deriv_root = op.join(bids_root, 'derivatives')
project_root = op.join(deriv_root, 'Day1')
fs_subjects_dir = op.join(deriv_root,'freesurfer','subjects')

In [ ]:
# pick a subject

subject = 'ON03748'
fs_subject = 'sub-' + subject
data_dict = nih2mne.utilities.bids_helpers.get_mri_dict(subject,bids_root, task='airpuff')

In [ ]:
# load in all the MRI stuff

bem = data_dict['bem'].load()
fwd = data_dict['fwd'].load()
src = fwd['src']
trans = data_dict['trans'].load()

In [ ]:
#Load the Raw data
bids_path = BIDSPath(root=bids_root, subject=subject, task='airpuff', run='01',session='01')
raw = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
# put a bandpass on the raw, and a notch
raw.filter(0.3, 50, n_jobs=n_jobs)
raw.notch_filter(freqs=[60])
# also make some filtered data in individual bands for later
theta=raw.copy().filter(4,8)
alpha=raw.copy().filter(8,12)
gamma=raw.copy().filter(30,50)

In [ ]:
# We also want to load in the empty room dataset so we can calculate a noise covariance 
bids_path = BIDSPath(root=bids_root, subject=subject, task='noise', run='01',session='01')
noise = mne.io.read_raw_ctf(bids_path.fpath, clean_names=True, preload=True, verbose=False)
nosie_alpha = noise.copy().filter(8,12)

In [ ]:
# Now lets extract the events, make epochs
evts, evtsid = mne.events_from_annotations(raw)
epochs = mne.Epochs(raw, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)

# make the filtered epochs as well
epochs_alpha = mne.Epochs(alpha, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
epochs_theta = mne.Epochs(theta, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)
epochs_gamma = mne.Epochs(gamma, evts, evtsid, tmin=-0.1, tmax=0.2, preload=True)

# let's also average the epochs to visualize the evoked responses
evk_stim = epochs['stim'].average()
evk_missingstim = epochs['missingstim'].average()

In [ ]:
%matplotlib inline
_=evk_stim.plot()
_=evk_missingstim.plot()

In [ ]:
cov_all = mne.compute_covariance(epochs, tmin=-0.1, tmax = 0.2, n_jobs=n_jobs)

In [ ]:
noise_cov = mne.make_ad_hoc_cov(raw.info)

In [ ]:
# Let's say we want to project the evoked response into source space
# notice here that I'm using the covariance from ALL the marks, not just the stimuli
filters = mne.beamformer.make_lcmv(evk_stim.info, fwd, cov_all, reg=0.05, pick_ori='max-power',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')

In [ ]:
stc=mne.beamformer.apply_lcmv(evk_stim, filters)

In [ ]:
brain=stc.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# You probably noticed the dreaded Beamformer Sign Ambiguity 

In [ ]:
# what happens if we use the normals from the freesurfer cortical surface? 
fwd_src_ori = mne.convert_forward_solution(fwd, surf_ori=True)
filters_src_ori = mne.beamformer.make_lcmv(evk_stim.info, fwd_src_ori, cov_all, reg=0.05, pick_ori='normal',
                                   noise_cov = noise_cov, weight_norm='unit-noise-gain')
stc_src_ori=mne.beamformer.apply_lcmv(evk_stim, filters)

In [ ]:
brain=stc_src_ori.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# You might *think* that looks worse, but now, the sign of the output is following the surface. 
# In opposing sulci, the surface normals are oriented opposite eachother. 

In [ ]:
# Frequently what we do is to invoke a sign "flip"

In [ ]:
# I've figured out that index 166 corresponds to roughly the peak of the evoked response.
# There are 8196 vertices
for i in range(8196):
    if stc.data[i,166] < 0:
        stc.data[i,:] *= -1
# You need to remember here, however, that you are also flipping vertices that aren't particularly active
# so you'll also be ensuring that all the noise in that time point is positive.

In [ ]:
brain=stc.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# We can make the time course for the missing stim as well
stc_missing=mne.beamformer.apply_lcmv(evk_missingstim, filters)

In [ ]:
# plot it 
brain=stc_missing.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# interesting, there does seem to be something out around 150ms, doesn't there.... but again the flip thing...

In [ ]:
# Remember our time frequency plots for this person - wasn't there something in alpha around that time? 
# Maybe we should look at alpha power

In [ ]:
cov_all_alpha = mne.compute_covariance(epochs_alpha, tmin=0.1, tmax = 0.2, n_jobs=n_jobs)
cov_stim_alpha = mne.compute_covariance(epochs_alpha['stim'], tmin=.1, tmax = 0.2, n_jobs=n_jobs)
cov_missingstim_alpha = mne.compute_covariance(epochs_alpha['missingstim'], tmin=0.1, tmax = 0.2, n_jobs=n_jobs)

In [ ]:
filters_alpha = mne.beamformer.make_lcmv(epochs_alpha.info, fwd, cov_all_alpha, reg=0.05, noise_cov=noise_cov)

In [ ]:
stc_stim_alpha = mne.beamformer.apply_lcmv_cov(cov_stim_alpha, filters_alpha)
stc_missing_alpha = mne.beamformer.apply_lcmv_cov(cov_missingstim_alpha, filters_alpha)

In [ ]:
stc_contrast.data=stc_stim_alpha.data/stc_missing_alpha.data

In [ ]:
brain=stc_contrast.plot(hemi='both', subjects_dir=fs_subjects_dir, subject=fs_subject)

In [ ]:
# Okay, now lets loop over all the subjects!!
subjects=['ON02811','ON03748','ON22671','ON42107','ON52662','ON61373','ON62003','']